In [49]:
import json
from yargy import Parser, rule, and_, or_
from yargy.predicates import gram, is_capitalized, dictionary, in_, normalized, eq, caseless, type
from yargy.interpretation import fact

CRAWLED_DATA = 'members.jl'

In [50]:
# how to extract job description from the crawled data
# table columns order: [marker, preposition, date, position, source, unit]

texts = []
with open(CRAWLED_DATA) as f:
    for line in f:
        j = json.loads(line)
        for row in j['table']:
            texts.append(row[3])

In [23]:
data = ['[сотр.] ОО НКВД 2 отдельной Краснознаменной армии',
 'врио нач. УПВО НКВД Грузинской ССР',
 'врио нач. спецтюрьмы ГУГБ НКВД СССР',
 'делопроизводитель-машинистка ОКР СМЕРШ 10 фронтового управления оборонительного строительства',
 'зам. министра внутренних дел УССР',
 'зам. нач. 3 СО Астраханского ГО УНКВД Сталинградской обл.',
 'зам. нач. 4 отдела УНКВД Харьковской обл.',
 'зам. нач. Актюбинского ИТЛ',
 'зам. нач. политотдела совхоза «Нарым» НКВД Узбекской ССР',
 'инспектор УНКВД Ленинградской обл.',
 'нач. Волховского РО УНКВД Ленинградской обл.',
 'нач. Змеиногорского РО УНКВД Алтайского края',
 'нач. Майнского РО УНКВД Куйбышевской обл.',
 'нач. ОКР СМЕРШ 86 стрелковой дивизии',
 'нач. ОО НКВД 227 авиадивизии',
 'нач. Осьминского РО УНКВД Ленинградской обл.',
 'нач. Спасского РО УНКВД Рязанской обл.',
 'нач. Халтуринского РО УНКВД Кировской обл.',
 'нач. оперпункта ст. Котовск ДТО НКВД Одесской железной дороги',
 'нач. отделения ОИТК и ТП УНКВД Чкаловской обл.',
 'нач. отделения ст. Куйбышевка-Восточная ДТО НКВД Амурской железной дороги',
 'нач. охраны тыла 20 армии Западного фронта',
 'нач. хозотделения 3 СО НКВД СССР',
 'нач. штаба 159 полка УПВО НКВД УССР',
 'нач. штаба ПВ НКВД Киевского округа',
 'оперуполномоченный 4 отделения ОО НКВД ЗабВО',
 'оперуполномоченный Балашовского РО УНКВД Саратовской обл.',
 'оперуполномоченный Кодымского РО УГБ НКВД УССР',
 'оперуполномоченный ОО НКВД Юго-Западного фронта',
 'оперуполномоченный ст. Куйбышевка ДТО НКВД Амурской железной дороги',
 'оперуполномоченный ст. Оренбург ДТО НКВД',
 'оперуполномоченный управления коменданта Московского Кремля НКВД СССР',
 'пом. нач. 3 отдела УНКВД Иркутской обл.',
 'пом. нач. 6 отдела УГБ НКВД Татарской АССР',
 'пом. нач. ОКР СМЕРШ 19 стрелкового корпуса',
 'пом. нач. Херсонского ГО УНКВД Николаевской обл. УССР',
 'пом. нач. отделения ст. Медведево ДТО НКВД Калининской железной дороги',
 'пом. нач. секретариата НКВД СССР',
 'пом. оперуполномоченного Муслюмовского РО УГБ НКВД Татарской АССР',
 'пом. оперуполномоченного ОО НКВД 13 казачьей кавалерийской дивизии УНКВД Азово-Черноморского края',
 'пом. оперуполномоченного УНКВД Актюбинской обл. Казахской ССР',
 'председатель отдела Военного трибунала УПВО НКВД по Донецкой обл.',
 'сотр. ОКР СМЕРШ 57 армии',
 'сотр. ТО НКВД Западной железной дороги',
 'сотрудник НКВД УССР',
 'сотрудник для особых поручений ОК НКВД СССР',
 'сотрудник контрразведки',
 'ст. оперуполномоченный 2 отделения ОО НКВД Юго-Западного фронта',
 'ст. оперуполномоченный 4 отделения ОО НКВД Волховского фронта',
 'ст. оперуполномоченный УНКВД Адыгейской АО',
 'ст. пом. нач. Особой инспекции Союзной контрольной комиссии в Румынии']

In [64]:
Record = fact(
    'Record',
    ['position', 'unit', 'region']
)

In [67]:
DOT = eq('.')

POSITION = rule(    
    eq('врид').optional(),
    or_(rule('пом', DOT.optional()),
        rule('зам', DOT.optional()),
        rule('ст', DOT.optional())).optional(),
    normalized('полковой').optional(),
    or_(
        rule(dictionary({'оперуполномоченный', 'сотрудник', 'командир', 'уполномоченный'})),
        rule(caseless('нач'), DOT)))

############################################

CITY = rule(eq('г'), DOT.optional(), gram('NOUN'))
SSR = or_(rule(gram('ADJF'), 
               or_(eq('ССР'), eq('АССР'), eq('АО'))),
          rule(or_(eq('БССР'), eq('УССР'))),
          rule(eq('СССР')))

KRAI = rule(gram('ADJF'), 
            or_(rule(normalized('край')),
                rule('обл', DOT.optional())))

OKRUG = rule(gram('ADJF'), normalized('округ'))

ARMY = rule(type('INT'), gram('ADJF').optional(), gram('ADJF').optional(), normalized('армия'))

FRONT = rule(gram('ADJF'), normalized('фронт'))

GUARD = rule(eq('гв'), DOT)
DIVISION = rule(type('INT'), GUARD.optional(), normalized('стрелковая').optional(), 
                or_(normalized('дивизия'),
                    normalized('авиадивизия'),
                    normalized('корпус')))

JD = or_(rule(eq('ж'), DOT, eq('д'), DOT),
         rule(normalized('железная'), normalized('дорога')))
RAILS = or_(rule(gram('ADJF'), JD),
            rule(JD, BY_NAME))

BY_NAME = rule(eq('им'), DOT, gram('NOUN'))

REGION = or_(
    CITY, SSR, KRAI, ARMY, DIVISION, RAILS, OKRUG, FRONT
)

#########################################

ST = rule(eq('ст'), DOT)
STATION = rule(ST, gram('NOUN'), eq('ДТО').optional(), eq('НКВД').optional())  
DEPARTMENT = rule(type('INT'), normalized('отдел'))

UNIT = or_(DEPARTMENT, STATION)

##########################################

## TODO fix code below
# RECORD = rule(
#     POSITION.interpretation(
#         Record.position
#     ),
#     UNIT.interpretation(
#         Record.unit
#     ),
#     REGION.interpretation(
#         Record.region
#     )
# ).interpretation(
#     Record
# )


pos_parser = Parser(POSITION)
unit_parser = Parser(UNIT)
region_parser = Parser(REGION)


In [70]:
for d in data:
    print('>>>', d)
    for match in pos_parser.findall(d):
        b, e = match.tokens[0].span[0], match.tokens[-1].span[1]
        print(f'Position: {d[b:e]}')
        
    for match in unit_parser.findall(d):
        b, e = match.tokens[0].span[0], match.tokens[-1].span[1]
        print(f'Unit: {d[b:e]}')
       
    for match in region_parser.findall(d):
        b, e = match.tokens[0].span[0], match.tokens[-1].span[1]
        print(f'Region: {d[b:e]}')
               
#     for match in parser.findall(d):
#         b, e = match.tokens[0].span[0], match.tokens[-1].span[1]
#         print(f'{(b, e)}: {d[:b]} [{d[b:e]}] {d[e+1:]}')

>>> [сотр.] ОО НКВД 2 отдельной Краснознаменной армии
Region: 2 отдельной Краснознаменной армии
>>> врио нач. УПВО НКВД Грузинской ССР
Position: нач.
Region: Грузинской ССР
>>> врио нач. спецтюрьмы ГУГБ НКВД СССР
Position: нач.
Region: СССР
>>> делопроизводитель-машинистка ОКР СМЕРШ 10 фронтового управления оборонительного строительства
>>> зам. министра внутренних дел УССР
Region: УССР
>>> зам. нач. 3 СО Астраханского ГО УНКВД Сталинградской обл.
Position: зам. нач.
Region: Сталинградской обл.
>>> зам. нач. 4 отдела УНКВД Харьковской обл.
Position: зам. нач.
Unit: 4 отдела
Region: Харьковской обл.
>>> зам. нач. Актюбинского ИТЛ
Position: зам. нач.
>>> зам. нач. политотдела совхоза «Нарым» НКВД Узбекской ССР
Position: зам. нач.
Region: Узбекской ССР
>>> инспектор УНКВД Ленинградской обл.
Region: Ленинградской обл.
>>> нач. Волховского РО УНКВД Ленинградской обл.
Position: нач.
Region: Ленинградской обл.
>>> нач. Змеиногорского РО УНКВД Алтайского края
Position: нач.
Region: Алтайского 

In [22]:
set(positions[:100])

{None,
 '[сотр.] ОО НКВД 2 отдельной Краснознаменной армии',
 'врио нач. УПВО НКВД Грузинской ССР',
 'врио нач. спецтюрьмы ГУГБ НКВД СССР',
 'делопроизводитель-машинистка ОКР СМЕРШ 10 фронтового управления оборонительного строительства',
 'зам. министра внутренних дел УССР',
 'зам. нач. 3 СО Астраханского ГО УНКВД Сталинградской обл.',
 'зам. нач. 4 отдела УНКВД Харьковской обл.',
 'зам. нач. Актюбинского ИТЛ',
 'зам. нач. политотдела совхоза «Нарым» НКВД Узбекской ССР',
 'инспектор УНКВД Ленинградской обл.',
 'нач. Волховского РО УНКВД Ленинградской обл.',
 'нач. Змеиногорского РО УНКВД Алтайского края',
 'нач. Майнского РО УНКВД Куйбышевской обл.',
 'нач. ОКР СМЕРШ 86 стрелковой дивизии',
 'нач. ОО НКВД 227 авиадивизии',
 'нач. Осьминского РО УНКВД Ленинградской обл.',
 'нач. Спасского РО УНКВД Рязанской обл.',
 'нач. Халтуринского РО УНКВД Кировской обл.',
 'нач. оперпункта ст. Котовск ДТО НКВД Одесской железной дороги',
 'нач. отделения ОИТК и ТП УНКВД Чкаловской обл.',
 'нач. отде